In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.affinity import translate, rotate
import math

# Define ChristmasTree class (same as before)
class ChristmasTree:
    def __init__(self, center=(0, 0), angle=0):
        self.center = center
        self.angle = angle
        tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
        tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
        x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
        y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
        self.vertices = list(zip(x, y))
        self.poly = Polygon(self.vertices)
        self.update_poly()

    def update_poly(self):
        self.poly = Polygon(self.vertices)
        self.poly = rotate(self.poly, self.angle, origin=(0, 0), use_radians=False)
        self.poly = translate(self.poly, self.center[0], self.center[1])

    def set_pose(self, x, y, angle):
        self.center = (x, y)
        self.angle = angle
        self.update_poly()

    def get_poly(self):
        return self.poly

def parse_submission(file_path):
    df = pd.read_csv(file_path)
    for col in ['x', 'y', 'deg']:
        df[col] = df[col].astype(str).str.replace('s', '').astype(float)
    return df

# Load failed submission
failed_sub = parse_submission('/home/submission/submission_sa_refined.csv')
failed_sub['N'] = failed_sub['id'].apply(lambda x: int(x.split('_')[0]))

# Check group 005 (N=5)
n = 5
group = failed_sub[failed_sub['N'] == n]
trees = []
for _, row in group.iterrows():
    trees.append(ChristmasTree((row['x'], row['y']), row['deg']))

# Check overlaps
overlap_found = False
for i in range(len(trees)):
    for j in range(i + 1, len(trees)):
        if trees[i].get_poly().intersects(trees[j].get_poly()):
            area = trees[i].get_poly().intersection(trees[j].get_poly()).area
            if area > 1e-9: # Tolerance check
                print(f"Overlap between {i} and {j}: Area={area:.10f}")
                overlap_found = True

if not overlap_found:
    print("No overlap found with standard tolerance.")
else:
    print("Overlap confirmed.")


No overlap found with standard tolerance.
